# Aufgabe

In dieser Aufgabe soll eine Künstliche Intelligenz für verschiedene Spiele entwickelt werden. Dabei soll ein objektorientierter Ansatz gewählt werden. Die Spiele werden im Folgenden genauer vorgestellt. Zusätzlich wird eine weitere KI zur Verfügung gestellt, gegen die gespielt und im besten Fall gewonnen werden soll.

## Vorbereitungen
Die Spielersymbole sollen in einem Enum gespeichert werden. Mögliche Werte sind X und O.

In [1]:
enum class Spieler{
    X, O
}

Alle Künstliche Intelligenzen sollen eine gemeinsame, generische Schnittstelle implementieren, die eine Methode `macheZug` definiert. Diese soll in den Klassen genauer ausgeführt werden. Als Parameter wird der Spielstand übergeben, welcher in der Methode aktualisiert und wieder zurückgegeben wird. Der generische Parameter ist sie Repräsentation des Spielstandes, da diese von Spiel zu Spiel verschieden ist.

In [2]:
interface KI<K> {
    fun macheZug (daten: K) : K
    fun gibSymbol () : Spieler
} 

Alle Spiele sollen von einer abstrakten, ebenfalls generischen Klasse `Spiel` erben. Diese besitzt 2 Felder für die beiden Spieler vom Typ `KI`. Außerdem sollen 2 abstrakte Methoden definiert werden:
* `testeGewonnen()`: Testet, ob ein Spieler gewonnen hat oder ein Unentschieden vorliegt. Zurückgegeben soll ein Paar, welches aus einem Wahrheitswert (ist das Spiel zu Ende?) und dem Spieler, der gewonnen hat (falls einer vorhanden ist, ansonsten `null`).
* `gibAktuellenStandAus()`: Gibt den aktuellen Spielstand aus.
* `spielen()`: Simuliert ein Spiel zwischen den beiden Künstlichen Intelligenzen. Die beiden Spieler machen solange einen Zug, bis ein Endzustand erreicht wurde. Spieler 1 fängt standardmäßig an.

In [3]:
abstract class Spiel<T, K> (val spieler1 : KI<K>, val spieler2 : KI<K>) {
    abstract var spielfeld: T
    abstract fun testeGewonnen() : Pair<Boolean, Spieler?> 
    abstract fun reset()
    abstract fun gibAktuellenStandAus()
    abstract fun transformSpielfeld(daten: K, spieler: Spieler) : T
    abstract fun gibSpielfeld(spieler: Spieler) : K
    fun spielen(){
        var aktuellerSpieler = spieler1
        var zuege = 0
        var test: Pair<Boolean, Spieler?>
        do{
            spielfeld = transformSpielfeld(aktuellerSpieler.macheZug(gibSpielfeld(aktuellerSpieler.gibSymbol())), aktuellerSpieler.gibSymbol())
            test = testeGewonnen()
            if (aktuellerSpieler == spieler1)
                aktuellerSpieler = spieler2
            else
                aktuellerSpieler = spieler1
            zuege++
            gibAktuellenStandAus()
        }
        while(!test.first && zuege < 5)
        if (test.first){
            when(test.second){
                Spieler.X -> println("Spieler X hat gewonnen.")
                Spieler.O -> println("Spieler O hat gewonnen.")
                else -> println("Unentschieden!")
            }
        }
        
        gibAktuellenStandAus()
    }
    
    fun spielenDebug(anzahlZuege: Int){
        var aktuellerSpieler = spieler1
        var zuege = 0
        var test: Pair<Boolean, Spieler?>
        do{
            spielfeld = transformSpielfeld(aktuellerSpieler.macheZug(gibSpielfeld(aktuellerSpieler.gibSymbol())), aktuellerSpieler.gibSymbol())
            test = testeGewonnen()
            if (aktuellerSpieler == spieler1)
                aktuellerSpieler = spieler2
            else
                aktuellerSpieler = spieler1
            zuege++
            gibAktuellenStandAus()
        }
        while(!test.first && zuege < anzahlZuege)
        if (test.first){
            when(test.second){
                Spieler.X -> println("Spieler X hat gewonnen.")
                Spieler.O -> println("Spieler O hat gewonnen.")
                else -> println("Unentschieden!")
            }
        }
        
        gibAktuellenStandAus()
    }
    
    fun testSpielen(): Spieler?{
        var aktuellerSpieler = spieler1
        var test: Pair<Boolean, Spieler?>
        do{
            spielfeld = transformSpielfeld(aktuellerSpieler.macheZug(gibSpielfeld(aktuellerSpieler.gibSymbol())), aktuellerSpieler.gibSymbol())
            test = testeGewonnen()
            if (aktuellerSpieler == spieler1)
                aktuellerSpieler = spieler2
            else
                aktuellerSpieler = spieler1
        }
        while(!test.first)
        reset()
        return test.second
        
    }
}

## TicTacToe 3/5

In dieser Teilaufgabe soll das Spiel [TicTacToe](https://de.wikipedia.org/wiki/Tic-Tac-Toe) implementiert werden. 
* Die zugehörige Klasse soll von `Spiel` erben und zusätzlich ein Feld für das Spielfeld besitzen. Das Spielfeld soll als 2-dimensionales Array sein, das mit einem Wert des Enums `Spieler` oder `null` gefüllt werden kann. Dies ist auch der generische Datentyp für die Oberklasse, sowie der KI. Standardmäßig sollen alle Felder mit dem Wert `null` gefüllt werden.

In [4]:
class TicTacToe (spieler1: KI<Array<Array<Spieler?>>>, spieler2: KI<Array<Array<Spieler?>>>) : Spiel<Array<Array<Spieler?>>, Array<Array<Spieler?>>> (spieler1, spieler2){
    override var spielfeld = Array<Array<Spieler?>>(3) { Array<Spieler?>(3) { null } }
    
    override fun testeGewonnen() : Pair<Boolean, Spieler?>{
        var testspieler = Spieler.X
        // Iteriert die möglichen Spieler
        for (spielernummer in 0..1){
            if (spielernummer != 0)
                testspieler = Spieler.O
            //Testet die horizontalen und vertikalen Linien
            for (i in 0..2){
                if (spielfeld[i][0] == testspieler && spielfeld[i][1] == testspieler && spielfeld[i][2] == testspieler)
                    return Pair(true, testspieler)
                if (spielfeld[0][i] == testspieler && spielfeld[1][i] == testspieler && spielfeld[2][i] == testspieler)
                    return Pair(true, testspieler)
            }
            //Testet die Diagonalen
            if(spielfeld[0][0] == testspieler && spielfeld[1][1] == testspieler && spielfeld[2][2] == testspieler)
                return Pair(true, testspieler)
            if(spielfeld[2][0] == testspieler && spielfeld[1][1] == testspieler && spielfeld[0][2] == testspieler)
                return Pair(true, testspieler)
        }
        //Testet, ob ein Unentschieden vorliegt
        if (spielfeld.all { it.none { it == null } })
            return Pair(true, null)
        return Pair(false, null)
    }
    
    override fun gibAktuellenStandAus() {
        for (zeile in spielfeld){
            for (feld in zeile){
                if (feld != null)
                    print(feld)
                else
                    print('-')
            }
            println()
        }
    }
    
    override fun reset(){
        spielfeld = Array<Array<Spieler?>>(3) { Array<Spieler?>(3) { null } }
    }
    
    override fun transformSpielfeld(neues_spielfeld: Array<Array<Spieler?>>, spieler: Spieler) = neues_spielfeld
    
    override fun gibSpielfeld(spieler: Spieler) = spielfeld
}

Implementieren Sie im nächsten Schritt eine eigene KI mit dem Namen `SpielerKITicTacToe`, die `BruteForceKITicTacToe` in einem direkten Duell schlägt. Ihr soll ebenfalls ein Wert des Enums `Spieler` übergeben werden, dass das Spielersymbol repräsentiert.

In [5]:
class BruteForceKITicTacToe (val symbol : Spieler) : KI<Array<Array<Spieler?>>> {
    override fun macheZug (daten: Array<Array<Spieler?>>) : Array<Array<Spieler?>>{
        var zufallsfeld = 0
        do {
            zufallsfeld = (0..8).random()
        }
        while (daten[zufallsfeld / 3][zufallsfeld - 3 * (zufallsfeld / 3)] != null)
        daten[zufallsfeld / 3][zufallsfeld - 3 * (zufallsfeld / 3)] = symbol
        return daten
    }
    
    override fun gibSymbol() = symbol
}

In [6]:
//Beispiellösung
class SpielerKITicTacToe (val symbol : Spieler) : KI<Array<Array<Spieler?>>> {
     override fun macheZug (daten: Array<Array<Spieler?>>) : Array<Array<Spieler?>>{
         val spielfeld = daten
         var gegnerSymbol = Spieler.X
         if (symbol == Spieler.X)
             gegnerSymbol = Spieler.O
         //Horizontale und Vertikale Niederlage Verhindern
         for(i in 0..2){
             if (spielfeld[i][0] == gegnerSymbol && spielfeld[i][1] == gegnerSymbol && spielfeld[i][2] != symbol){
                 spielfeld[i][2] = symbol
                 return spielfeld
             }
             if (spielfeld[i][1] == gegnerSymbol && spielfeld[i][2] == gegnerSymbol && spielfeld[i][0] != symbol){
                 spielfeld[i][0] = symbol
                 return spielfeld
             }
             if (spielfeld[i][0] == gegnerSymbol && spielfeld[i][2] == gegnerSymbol && spielfeld[i][1] != symbol){
                 spielfeld[i][1] = symbol
                 return spielfeld
             }
             if (spielfeld[0][i] == gegnerSymbol && spielfeld[1][i] == gegnerSymbol && spielfeld[2][i] != symbol){
                 spielfeld[2][i] = symbol
                 return spielfeld
             }
             if (spielfeld[1][i] == gegnerSymbol && spielfeld[2][i] == gegnerSymbol && spielfeld[0][i] != symbol){
                 spielfeld[0][i] = symbol
                 return spielfeld
             }
             if (spielfeld[0][i] == gegnerSymbol && spielfeld[2][i] == gegnerSymbol && spielfeld[1][i] != symbol){
                 spielfeld[1][i] = symbol
                 return spielfeld
             }
         }
         //Diagonale Niederlage Verhindern
         if (spielfeld[0][0] == gegnerSymbol && spielfeld[1][1] == gegnerSymbol && spielfeld[2][2] != symbol){
             spielfeld[2][2] = symbol
             return spielfeld
         }
         if (spielfeld[1][1] == gegnerSymbol && spielfeld[2][2] == gegnerSymbol && spielfeld[0][0] != symbol){
             spielfeld[0][0] = symbol
             return spielfeld
         }
         if (spielfeld[0][0] == gegnerSymbol && spielfeld[2][2] == gegnerSymbol && spielfeld[1][1] != symbol){
             spielfeld[1][1] = symbol
             return spielfeld
         }
         if (spielfeld[0][2] == gegnerSymbol && spielfeld[1][1] == gegnerSymbol && spielfeld[2][0] != symbol){
             spielfeld[2][0] = symbol
             return spielfeld
         }
         if (spielfeld[1][1] == gegnerSymbol && spielfeld[2][0] == gegnerSymbol && spielfeld[0][2] != symbol){
             spielfeld[0][2] = symbol
             return spielfeld
         }
         if (spielfeld[0][2] == gegnerSymbol && spielfeld[2][0] == gegnerSymbol && spielfeld[1][1] != symbol){
             spielfeld[1][1] = symbol
             return spielfeld
         }
         //Ansonsten wird ein Zufallsfeld gewählt
        var zufallsfeld = 0
        do {
            zufallsfeld = (0..8).random()
        }
        while (spielfeld[zufallsfeld / 3][zufallsfeld - 3 * (zufallsfeld / 3)] != null)
        spielfeld[zufallsfeld / 3][zufallsfeld - 3 * (zufallsfeld / 3)] = symbol
        return spielfeld
            
     }
     
     override fun gibSymbol() = symbol
}

In [7]:
//Test

var spielerKI = 0
var bruteForceKI = 0
var unentschieden = 0

val ki1 = SpielerKITicTacToe(Spieler.X)
val ki2 = BruteForceKITicTacToe(Spieler.O)

val spiel1 = TicTacToe(ki1, ki2)
for(i in 0..100){
    when(spiel1.testSpielen()){
        Spieler.X -> spielerKI++
        Spieler.O -> bruteForceKI++
        else -> unentschieden++
    }
}

val spiel2 = TicTacToe(ki2, ki1)
for(i in 0..100){
    when(spiel2.testSpielen()){
        Spieler.X -> bruteForceKI++
        Spieler.O -> spielerKI++
        else -> unentschieden++
    }
}

println("Anzahl Siege SpielerKI: $spielerKI")
println("Anzahl Siege BruteForceKI: $bruteForceKI")
println("Anzahl Unentschieden: $unentschieden")


if(spielerKI > bruteForceKI)
    println("Super! Deine KI hat öfter gewonnen!")
else{
    println("Deine KI hat leider öfter verloren.")
}


Anzahl Siege SpielerKI: 92
Anzahl Siege BruteForceKI: 45
Anzahl Unentschieden: 65
Super! Deine KI hat öfter gewonnen!


## Windige Wanderung! 4/5
Dieses Spiel ähnelt dem bekannten Spiel "Mensch ärger dich nicht". Zwei Spieler:innen versuchen ihre Spielfiguren (je 4) in das Ziel zu bewegen. Vom Start zum Ziel gibt es 20 Felder. Die Spielfelder der beiden Spieler:innen sind voneinander getrennt. Die Spielfiguren werden nach den Augenzahl eines Würfels gezogen. Falls eine 6 gewürfelt wurde, kann eine neue Spielfigur auf das Feld 0 gestellt werde. Es kann aber auch eine beliegebige bereits auf dem Feld befindliche Figur weitergezogen werden. Auf jedem Feld kann nur eine Spielfigur stehen. Sollte es zu einem Konflikt kommen, wird die Figur ein Feld weiter platziert. <br /> Aufgrund der räumlichen Trennung der Spielfelder, können sich die Spielfiguren nicht gegenseitig "rauswerfen". Dafür sorgt jedoch der Wind. Nach jedem Zug werden zufällig 2 der 20 Felder getroffen, sodass die Spielfiguren wieder vom Anfang starten müssen. Die Spielfiguren können aber vor dem Wind geschützt werden. Die Felder 1,4,8,12,16 und 20 sind windgeschützt und somit sicher. Gewonnen hat die Person dessen Figuren alle das Ziel erreicht haben.<br /><br />
Im ersten Schritt soll eine Klasse `WindigeWanderung` implementiert werden, die von der Klasse `Spiel` erbt. Das Spielfeld ist ein 2-dimensionales Array, das die Größe 2x20 besitzt. Zusätzlich soll für jeden Spieler gespeichert werden, wieviele Spielfiguren noch am Start warten und bereits im Ziel sind.
Vervollständigen Sie des Weiteren alle Methoden, die die Klasse `Spiel` vorgibt.

In [16]:
class WindigeWanderung (spieler1: KI<Pair<Array<Spieler?>, Array<Int>>>, spieler2: KI<Pair<Array<Spieler?>, Array<Int>>>) : Spiel<Array<Array<Spieler?>>, Pair<Array<Spieler?>, Array<Int>>> (spieler1, spieler2){
    override var spielfeld = Array<Array<Spieler?>>(2) { Array<Spieler?>(20) { null } }
    var figuren = arrayOf( arrayOf(4,0), arrayOf(4,0))
    
    override fun testeGewonnen(): Pair<Boolean, Spieler?>{
        if (figuren[0][1] == 4)
            return Pair(true, Spieler.X)
        else if (figuren[1][1] == 4)
            return Pair(true, Spieler.O)
        return Pair(false, null)
    }
    
    override fun reset() {
        spielfeld = Array<Array<Spieler?>>(2) { Array<Spieler?>(20) { null } }
        figuren = arrayOf( arrayOf(4,0), arrayOf(4,0))
    } 
    
    override fun gibAktuellenStandAus() {
        for (zeile in spielfeld){
            for (feld in zeile){
                if (feld != null)
                    print(feld)
                else
                    print('-')
            }
            println()
        }
        println("Spieler1: Start ${figuren[0][0]}, Ende ${figuren[0][1]}")
        println("Spieler2: Start ${figuren[1][0]}, Ende ${figuren[1][1]}")
        println()
    }
    
    private fun winden(){
        var feld1 = (0..19).random()
        var feld2 = (0..19).random()
        while (feld1 in 0..19 step 4 && feld2 in 0..19 step 4){
            feld1 = (0..19).random()
            feld2 = (0..19).random()
        }
        
        if(spielfeld[0][feld1] == Spieler.X){
            spielfeld[0][feld1] = null
            figuren[0][0] += 1
        }
        else if(spielfeld[1][feld1] == Spieler.O){
            spielfeld[1][feld1] = null
            figuren[1][0] += 1
        }
        if(spielfeld[0][feld2] == Spieler.X){
            spielfeld[0][feld2] = null
            figuren[0][0] += 1
        }
        else if(spielfeld[1][feld2] == Spieler.O){
            spielfeld[1][feld2] = null
            figuren[1][0] += 1
        }
    }
    
    override fun transformSpielfeld(daten: Pair<Array<Spieler?>, Array<Int>>, spieler: Spieler): Array<Array<Spieler?>> {
        if (spieler == Spieler.X){
            spielfeld[0] = daten.first
            figuren[0] = daten.second
        }
        else{
            spielfeld[1] = daten.first
            figuren[1] = daten.second
        } 
        winden()
        return spielfeld
    }
    
    override fun gibSpielfeld(spieler: Spieler): Pair<Array<Spieler?>, Array<Int>> {
        return if (spieler == Spieler.X)
            Pair(spielfeld[0], figuren[0])
        else
            Pair(spielfeld[1], figuren[1])
    }
}

In [17]:
class BruteForceKIWindigeWanderung (val symbol: Spieler) : KI<Pair<Array<Spieler?>, Array<Int>>>{
    override fun macheZug(daten: Pair<Array<Spieler?>, Array<Int>>): Pair<Array<Spieler?>, Array<Int>>{
        val spielfeld = daten.first
        val figuren = daten.second
        val augenzahl = wuerfeln()
        //Keine Figur auf dem Feld
        if(figuren[0] == 4){
            if (augenzahl == 6 && spielfeld[0] == null){
                spielfeld[0] = symbol
                figuren[0] -= 1
            }
            return Pair(spielfeld, figuren)
        }
        //Es ist eine Figur auf dem Spielfeld
        else{
            //Figur rausstellen falls möglich
            if (augenzahl == 6 && spielfeld[0] == null && figuren[0] < 4 && figuren[0] > 0){
                spielfeld[0] = symbol
                figuren[0] -= 1
                return Pair(spielfeld, figuren)
            }
            else{
                var indexLetzteFigur = 0
                for(i in 19 downTo 0){
                    if(spielfeld[i] == symbol){
                        indexLetzteFigur = i
                        break
                    }
                }
                
                if((indexLetzteFigur + augenzahl)>19){
                    spielfeld[indexLetzteFigur] = null
                    figuren[1] += 1
                }
                else{
                    spielfeld[indexLetzteFigur] = null
                    spielfeld[indexLetzteFigur + augenzahl] = symbol
                }
                return Pair(spielfeld, figuren)
            }
        }
    }
    
    private fun wuerfeln() = (1..6).random()
    
    override fun gibSymbol() = symbol
}

In [18]:
class SpielerKIWindigeWanderung (val symbol: Spieler) : KI<Pair<Array<Spieler?>, Array<Int>>>{
    override fun macheZug(daten: Pair<Array<Spieler?>, Array<Int>>): Pair<Array<Spieler?>, Array<Int>>{
        val spielfeld = daten.first
        val figuren = daten.second
        val augenzahl = wuerfeln()
        //Keine Figur auf dem Feld
        if(spielfeld.count { it == symbol } <= 2 && augenzahl == 6 && spielfeld[0] == null && figuren[0] > 0){
            spielfeld[0] = symbol
            figuren[0] -= 1
            return Pair(spielfeld, figuren)
        }
        //Es ist eine Figur auf dem Spielfeld
        else{
            //Figur sichern falls möglich
            var indexFigur = 0
            var gesichert = false
            for(i in 19 downTo 0){
                if(spielfeld[i] == symbol && (i+augenzahl) in 0..19 step 4){
                    spielfeld[i] = null
                    spielfeld[i+augenzahl] = symbol
                    gesichert = true
                    break
                }
            }
            if(gesichert)
                return Pair(spielfeld, figuren)
            
            //Figur rausstellen falls möglich
            if (augenzahl == 6 && spielfeld[0] == null && figuren[0] < 4 && figuren[0] > 0){
                spielfeld[0] = symbol
                figuren[0] -= 1
                return Pair(spielfeld, figuren)
            }
            else{
                var indexLetzteFigur = 0
                for(i in 19 downTo 0){
                    if(spielfeld[i] == symbol){
                        indexLetzteFigur = i
                        break
                    }
                }
                
                if((indexLetzteFigur + augenzahl)>19){
                    spielfeld[indexLetzteFigur] = null
                    figuren[1] += 1
                }
                else{
                    spielfeld[indexLetzteFigur] = null
                    spielfeld[indexLetzteFigur + augenzahl] = symbol
                }
                return Pair(spielfeld, figuren)
            }
        }
        
    }

    private fun wuerfeln() = (1..6).random()
    
    override fun gibSymbol() = symbol
}

In [19]:
//Test

var spielerKI = 0
var bruteForceKI = 0
var unentschieden = 0

val ki1 = SpielerKIWindigeWanderung(Spieler.X)
val ki2 = BruteForceKIWindigeWanderung(Spieler.O)

val spiel1 = WindigeWanderung(ki1, ki2)
for(i in 0..100){
    when(spiel1.testSpielen()){
        Spieler.X -> spielerKI++
        Spieler.O -> bruteForceKI++
        else -> unentschieden++
    }
}

val spiel2 = WindigeWanderung(ki2, ki1)
for(i in 0..100){
    when(spiel2.testSpielen()){
        Spieler.X -> bruteForceKI++
        Spieler.O -> spielerKI++
        else -> unentschieden++
    }
}

println("Anzahl Siege SpielerKI: $spielerKI")
println("Anzahl Siege BruteForceKI: $bruteForceKI")
println("Anzahl Unentschieden: $unentschieden")


if(spielerKI > bruteForceKI)
    println("Super! Deine KI hat öfter gewonnen!")
else{
    println("Deine KI hat leider öfter verloren.")
}


Anzahl Siege SpielerKI: 96
Anzahl Siege BruteForceKI: 106
Anzahl Unentschieden: 0
Deine KI hat leider öfter verloren.


## Zehntausend 1/5
Bei [Zehntausend](https://de.wikipedia.org/wiki/Zehntausend_(Spiel)https://de.wikipedia.org/wiki/Zehntausend_(Spiel)) handelt es sich um ein einfaches Würfelspiel. Es werden mindestens 2 Spieler (in dieser Aufgabe genau 2) und 5 Würfel benötigt. Es wird abwechselnd mit allen 5 Würfeln gewürfelt. Die wichtigen Augenzahlen sind 1 und 5. Während die 1 dem Spieler 100 Punkte auf seinem Konto hinzufügt, ist eine 5 nur 50 Punkte wert. Werden 3 1er gewürfelt ergeben diese 1000 Punkte, 4 1er sogar 10000. Bei 5ern ist dies analog anzuwenden. 3 Würfel mit der AUgenzahl 5 ergeben 500 Punkte, 4 hingegen 5000. Jedoch muss ein solcher "Pasch" mit einer weiteren 1 oder 5 im nächsten Wurf bestätigt werden. Ansonsten verfallen die gesammelten Punkte. Jeder Spieler kann nach seinem ersten Wurf, falls dieser Punkte erbringen würde, entscheiden, ob er noch einmal würfeln möchte. Falls dort wieder Punkte erzielt werden, werden die Punktzahlen der Würfe addiert. Ist dort keine Punkteausbeute möglich, verfallen jedoch alle gesammelten Punkte. Gewonnen hat der Spieler, der zuerst 10000 Punkte erreicht.

<table>
<thead>
  <tr>
    <th>Augenzahlen</th>
    <th>Punkte</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>1</td>
    <td>100</td>
  </tr>
  <tr>
    <td>5</td>
    <td>50</td>
  </tr>
  <tr>
    <td>1,5</td>
    <td>150</td>
  </tr>
  <tr>
    <td>1,1,5,5</td>
    <td>300</td>
  </tr>
  <tr>
    <td>1,1,1</td>
    <td>1000</td>
  </tr>
  <tr>
    <td>5,5,5</td>
    <td>500</td>
  </tr>
  <tr>
    <td>1,1,1,1</td>
    <td>10000</td>
  </tr>
  <tr>
    <td>5,5,5,5</td>
    <td>5000</td>
  </tr>
</tbody>
</table>

Im ersten Schritt soll das Spiel mit der Klasse `Zehntausend` implementiert werden. Dieses soll von `Spiel` erben. Die KIs würfeln eigenständig und bekommen als einzige Information ihren aktuellen Punktestand übergeben.

In [20]:
class Zehntausend (spieler1: KI<Int>, spieler2: KI<Int>) : Spiel<Array<Int>, Int> (spieler1, spieler2){
    override var spielfeld = arrayOf(0,0)
    
    override fun testeGewonnen(): Pair<Boolean, Spieler?>{
        if (spielfeld[0] >= 10000)
            return Pair(true, Spieler.X)
        else if (spielfeld[1] >= 10000)
            return Pair(true, Spieler.O)
        return Pair(false, null)
    }
    
    override fun reset() {
        spielfeld = arrayOf(0,0)
    } 
    
    override fun gibAktuellenStandAus() {
        println("Spieler1: ${spielfeld[0]}")
        println("Spieler2: ${spielfeld[1]}")
        println()
    }
    
    override fun transformSpielfeld(daten: Int, spieler: Spieler): Array<Int> {
        if (spieler == Spieler.X){
            spielfeld[0] += daten
        }
        else{
            spielfeld[1] += daten
        } 
        return spielfeld
    }
    
    override fun gibSpielfeld(spieler: Spieler): Int {
        return if (spieler == Spieler.X)
            spielfeld[0]
        else
            spielfeld[1]
    }
}

In [21]:
class BruteForceKIZehntausend (val symbol: Spieler) : KI<Int>{
    override fun macheZug(daten: Int): Int{
        var punkte = 0
        var bestaetigen = false
        
        do {
            val augenzahlen = wuerfeln()
            //Analyse
            val anzahl1 = augenzahlen.count { it == 1}
            val anzahl5 = augenzahlen.count { it == 5}

            if (bestaetigen && anzahl1 == 0 && anzahl5 == 0)
                return 0
            if (anzahl1 == 3){
                punkte += 1000 + anzahl5 * 50
                bestaetigen = true
            }
            else if (anzahl1 == 4){
                punkte += 10000 + anzahl5 * 50
                bestaetigen = true
            }
            else if (anzahl5 == 3){
                punkte += 500 + anzahl1 * 100
                bestaetigen = true
            }
            else if (anzahl5 == 4){
                punkte += 5000 + anzahl1 * 100
                bestaetigen = true
            }
            else{
                punkte += anzahl5 * 50 + anzahl1 * 100
                bestaetigen = false
            }
        } while(bestaetigen)
        return punkte
    }
    
    private fun wuerfeln() = listOf((1..6).random(), (1..6).random(), (1..6).random(), (1..6).random(), (1..6).random())
    
    override fun gibSymbol() = symbol
}

In [22]:
class SpielerKIZehntausend(val symbol: Spieler) : KI<Int>{
    override fun macheZug(daten: Int): Int{
        var punkte = 0
        var bestaetigen = false
        
        do {
            val augenzahlen = wuerfeln()
            //Analyse
            val anzahl1 = augenzahlen.count { it == 1}
            val anzahl5 = augenzahlen.count { it == 5}

            if (bestaetigen && anzahl1 == 0 && anzahl5 == 0)
                return 0
            if (anzahl1 == 3){
                punkte += 1000 + anzahl5 * 50
                bestaetigen = true
            }
            else if (anzahl1 == 4){
                punkte += 10000 + anzahl5 * 50
                bestaetigen = true
            }
            else if (anzahl5 == 3){
                punkte += 500 + anzahl1 * 100
                bestaetigen = true
            }
            else if (anzahl5 == 4){
                punkte += 5000 + anzahl1 * 100
                bestaetigen = true
            }
            else{
                punkte += anzahl5 * 50 + anzahl1 * 100
                if(punkte < 200)
                    bestaetigen = true
                else
                    bestaetigen = false
            }
        } while(bestaetigen)
        return punkte
    }
    
    private fun wuerfeln() = listOf((1..6).random(), (1..6).random(), (1..6).random(), (1..6).random(), (1..6).random())
    
    override fun gibSymbol() = symbol
}

In [23]:
//Test
var spielerKI = 0
var bruteForceKI = 0
var unentschieden = 0

val ki1 = SpielerKIZehntausend(Spieler.X)
val ki2 = BruteForceKIZehntausend(Spieler.O)

val spiel1 = Zehntausend(ki1, ki2)
for(i in 0..100){
    when(spiel1.testSpielen()){
        Spieler.X -> spielerKI++
        Spieler.O -> bruteForceKI++
        else -> unentschieden++
    }
}

val spiel2 = Zehntausend(ki2, ki1)
for(i in 0..100){
    when(spiel2.testSpielen()){
        Spieler.X -> bruteForceKI++
        Spieler.O -> spielerKI++
        else -> unentschieden++
    }
}

println("Anzahl Siege SpielerKI: $spielerKI")
println("Anzahl Siege BruteForceKI: $bruteForceKI")
println("Anzahl Unentschieden: $unentschieden")


if(spielerKI > bruteForceKI)
    println("Super! Deine KI hat öfter gewonnen!")
else{
    println("Deine KI hat leider öfter verloren.")
}


Anzahl Siege SpielerKI: 104
Anzahl Siege BruteForceKI: 98
Anzahl Unentschieden: 0
Super! Deine KI hat öfter gewonnen!
